In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch 
from src import AllModule

state_dict = torch.load('checkpoints/all-val_error=0.70873-epoch=3.ckpt')['state_dict']
model = AllModule(dict(backbone='resnet18', pretrained=True, mlp_layers=[256, 512], mlp_dropout=0.))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [3]:
from tqdm import tqdm
from src import AllDataModule, AllDataset
import albumentations as A

model.cuda()

tta_trans = [
    None,
    {'HorizontalFlip': {'p': 1}},
    {'VerticalFlip': {'p': 1}},
    {'Transpose': {'p': 1}},
    {'RandomRotate90': {'p': 1}},
]

dm = AllDataModule(batch_size=512, num_workers=10, pin_memory=True)
dm.setup()

all_labels = []
for r, trans in enumerate(tta_trans):
    dm.ds_test = AllDataset(
        dm.data_test.observation_id.values, dm.latlng_test, dm.bio_test, trans=A.Compose([
            getattr(A, trans)(**params) for trans, params in trans.items()
        ], additional_targets={'nir': 'image', 'alt': 'image', 'lc': 'image'})
        if trans is not None else None)
    labels, observations = [], []
    for batch in tqdm(dm.test_dataloader()):
        preds = model.predict(batch)
        values, ixs = preds.topk(30)
        labels += [' '.join([str(i.item()) for i in ix]) for ix in ixs]
        observation_ids = batch['observation_id']
        observations += observation_ids.cpu().numpy().tolist()
    all_labels.append(labels)

In [ ]:
final_labels

In [ ]:
import pandas as pd 

submission = pd.DataFrame({'Id': observations, 'Predicted': final_labels})
submission.to_csv('submission.csv', index=False)
submission.sample(5)

,Id,Predicted
11831,10157379,49 372 507 471 160 692 1091 397 880 171 192 68...
21630,20470861,5580 1762 9282 6856 6110 5565 6487 5940 6960 1...
2811,10737754,333 308 510 1009 259 1059 68 831 207 360 605 3...
13651,10154977,1009 523 207 340 565 587 402 338 342 416 816 1...
26516,21006144,1075 2159 129 201 6191 5366 770 6567 125 739 5...


In [ ]:
sample_submission = pd.read_csv('data/sample_submission.csv')
assert len(sample_submission) == len(submission)
sample_submission.sample(10)

,Id,Predicted
5077,10240778,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
25840,20935540,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
443,10020680,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
31170,21505725,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
32140,21607863,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
11667,10558925,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
14523,10694280,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
5730,10273678,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
29021,21273631,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2323,10111087,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
